In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,Dropout,Input,concatenate
from keras.optimizers import SGD
from keras import utils
from keras.datasets import cifar10

Using TensorFlow backend.


In [3]:
(trainX,trainY),(testX,testY)=cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [4]:
print('shape of trainX: ',trainX.shape,'shape of testX: ',testX.shape)
print('shape of trainY: ',trainY.shape,'shape of testY: ',testY.shape)

shape of trainX:  (50000, 32, 32, 3) shape of testX:  (10000, 32, 32, 3)
shape of trainY:  (50000, 1) shape of testY:  (10000, 1)


In [5]:
train_images = trainX
test_images = testX
train_labels = trainY
test_labels = testY

In [6]:
trainX = trainX / 255.0
testX = testX / 255.0
trainY = utils.to_categorical(trainY)
testY = utils.to_categorical(testY)

In [7]:
input_img = Input(shape = (32, 32, 3))

In [8]:
stem_1 = Conv2D(64,(7,7),activation='relu',kernel_initializer='he_uniform',padding='same')(input_img)
stem_1 = MaxPooling2D((3,3))(stem_1)
stem_1 = Conv2D(64,(1,1),activation='relu',kernel_initializer='he_uniform',padding='same')(stem_1)
stem_1 = Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same')(stem_1)
stem_1 = MaxPooling2D((3,3))(stem_1)

In [9]:
tower_1 = Conv2D(64,(1,1),activation='relu',kernel_initializer='he_uniform',padding='same')(stem_1)
tower_1 = Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same')(tower_1)
tower_2 = Conv2D(64,(1,1),activation='relu',kernel_initializer='he_uniform',padding='same')(stem_1)
tower_2 = Conv2D(64,(5,5),activation='relu',kernel_initializer='he_uniform',padding='same')(tower_2)
tower_3 = MaxPooling2D((3,3),padding='same',strides=(1,1))(stem_1)
tower_3 = Conv2D(64,(1,1),activation='relu',kernel_initializer='he_uniform',padding='same')(tower_3)

In [10]:
output = concatenate([tower_1,tower_2,tower_3],axis = 3)

In [15]:
output = Flatten()(output)
output = Dense(256,activation='relu',kernel_initializer='he_uniform')(output)

In [16]:
out = Dense(10,activation = 'softmax')(output)

In [17]:
model = Model(inputs = input_img, outputs = out)
opt = SGD(lr=0.01,momentum=0.9)
model.compile(optimizer=opt,metrics=['accuracy'],loss='categorical_crossentropy')

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 10, 10, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 10, 10, 64)   4160        max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [19]:
history = model.fit(trainX,trainY,batch_size=100,epochs=15,validation_data=(testX,testY),verbose=2)

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
 - 97s - loss: 1.7637 - accuracy: 0.3395 - val_loss: 1.5475 - val_accuracy: 0.4198
Epoch 2/15
 - 97s - loss: 1.4063 - accuracy: 0.4856 - val_loss: 1.3250 - val_accuracy: 0.5132
Epoch 3/15
 - 99s - loss: 1.2537 - accuracy: 0.5478 - val_loss: 1.2383 - val_accuracy: 0.5578
Epoch 4/15
 - 104s - loss: 1.1314 - accuracy: 0.5953 - val_loss: 1.1246 - val_accuracy: 0.6022
Epoch 5/15
 - 105s - loss: 1.0453 - accuracy: 0.6269 - val_loss: 1.1206 - val_accuracy: 0.6146
Epoch 6/15


KeyboardInterrupt: 

In [ ]:
predictions = model.predict(testX)
class_label = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
plt.figure(figsize=(25,25))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_images[i])
    plt.xlabel("{} {:2.0f}% ".format(class_label[np.argmax(predictions[i])], 100*np.max(predictions[i])))
    plt.title(class_label[int(test_labels[i])])
plt.show()

In [ ]:
test_labels[0]